### Import of Functions

In [ ]:
import os
import sys
print (os.getcwd())
sys.path.append('/home/Hanna.Drimalla/ml-autism/scripts')

In [ ]:
#import myreg
import myml
import mycharite

import pandas as pd
import matplotlib
import seaborn as sns
import numpy as np

# Main Code

In [ ]:
df, df_audio, action_r, action_c, gaze, audio = mycharite.load('charite')
print ('data loaded')

In [ ]:
df=mycharite.aq_charite(df)
df=mycharite.exclude_outlier(df) 
df=mycharite.adapt_times(df)
df=mycharite.ados_charite(df)

In [ ]:
print ('NT ' + str(sum(df.groupby('vpn').mean().asc==0))) #43
print ('ASC ' + str(sum(df.groupby('vpn').mean().asc==1))) #37

In [ ]:
df[['vpn', 'asc', 'sex']].groupby('vpn').mean().groupby('asc').describe()

In [ ]:
def calc_speed(df, var):

    epochs = pd.DataFrame()

    speeds = []
    accs = []
    
    for i, ((vpn, conversation), epoch) in enumerate(df.groupby(['vpn', 'conversation'])):
        for v in var:
            values = epoch.loc[:, var]

            speed = np.diff(values.values, axis=0) #erste Ableitung
            acc = np.diff(speed, axis=0) #zweite Ableitung
            speed = np.linalg.norm(speed, axis=1) #absolute Werte
            acc = np.linalg.norm(acc, axis=1)

            speeds.append(speed)
            accs.append(acc)

            epochs.loc[i, ('mean_speed_'+v)] = np.nanmean(speed)
            epochs.loc[i, ('mean_acc_'+v)] = np.nanmean(acc)
            epochs.loc[i, ('var_speed_'+v)] = np.nanvar(speed)
            epochs.loc[i, ('var_acc_'+v)] = np.nanvar(acc)

        epochs.loc[i, 'vpn'] = vpn
        epochs.loc[i, 'conversation'] = conversation

    speeds = pd.DataFrame(speeds)
    accs = pd.DataFrame(accs)
    speeds = pd.concat([speeds, epochs], axis=1)
    accs = pd.concat([accs, epochs], axis=1)
    
    return epochs

def calc_gaze_var(df):
    
    center = lambda x: (x - x.median()) 
    df['gaze_angle_y_centered']=df.groupby(['vpn'])['gaze_angle_y'].transform(center)
    center = lambda x: (x - x.median()) 
    df['gaze_angle_x_centered']=df.groupby(['vpn'])['gaze_angle_x'].transform(center)

    df['gaze_angle_x_centered_abs']=np.abs(df['gaze_angle_x_centered'])
    df['gaze_angle_y_centered_abs']=np.abs(df['gaze_angle_y_centered'])

    df['proband']=(df.conversation=='joy_proband')|(df.conversation=='ekel_proband')|(df.conversation=='neutral_proband')
    df['actress']=(df.conversation=='joy_speaker')|(df.conversation=='ekel_speaker')|(df.conversation=='neutral_speaker')
    df.loc[(df.conversation!='intro'), "whospeaks"] = df[['proband', 'actress']].idxmax(axis=1) 
    
    return df


var=['gaze_angle_x', 'gaze_angle_y']
df_speed_acc=calc_speed(df, var)
df=calc_gaze_var(df)
#df=df.groupby(['vpn', 'conversation']).mean()
df=pd.merge(df, df_speed_acc, on=['vpn', 'conversation'])

### Prediciton of Autism Diagnosis (Charite-Data)

### Gaze

In [ ]:
features=['gaze_angle_x', 'gaze_angle_y']
df, new_features=myml.calc_features(df, features)
new_features.append('sex')
X, y, vpn = myml.prepare_for_classify(df, new_features)
pr_rf_gaze, pr_svm_gaze, y_gaze, test_index_gaze=myml.classification(X, y, 'gaze')

## Face

In [ ]:
features=['AU06_r', 'AU12_r', 'AU04_r', 'AU09_r', 'AU15_r']
df, new_features=myml.calc_features(df, features)
new_features.append('sex')
X, y, vpn = myml.prepare_for_classify(df, new_features)
pr_rf_face, pr_svm_face, y_face, test_index_face=myml.classification(X, y, 'face')

In [ ]:
# hier für das gesamte Gesicht (alle AUs)
df, new_features=myml.calc_features(df, action_r)
new_features.append('sex')
X, y, vpn = myml.prepare_for_classify(df, new_features)
pr_rf_face_full, pr_svm_face_full, y_face_full, test_index_face_full=myml.classification(X, y, 'face_full')

### Audio

In [ ]:
audio=['spectrum_0', 'spectrum_1', 'spectrum_2', 'spectrum_3', 'spectrum_4', 'spectrum_5',
       'spectrum_6', 'spectrum_7', 'spectrum_8', 'spectrum_9', 'spectrum_10',
       'spectrum_11', 'spectrum_12', 'spectrum_13', 'spectrum_14',
       'spectrum_15', 'spectrum_16', 'spectrum_17', 'spectrum_18',
       'spectrum_19', 'spectrum_20', 'spectrum_21', 'spectrum_22',
       'spectrum_23', 'spectrum_24', 'spectrum_25', 'spectrum_26',
       'spectrum_27', 'spectrum_28', 'spectrum_29', 'spectrum_30',
       'spectrum_31', 'spectrum_32', 'spectrum_33', 'spectrum_34',
       'spectrum_35', 'spectrum_36', 'spectrum_37', 'spectrum_38',
       'spectrum_39', 'pitch', 'meanF0Hz',
       'stdevF0Hz', 'HNR', 'localJitter', 'localabsoluteJitter', 'rapJitter',
       'ppq5Jitter', 'ddpJitter', 'localShimmer', 'localdbShimmer',
       'apq3Shimmer', 'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer', 'JitterPCA',
       'ShimmerPCA']


df_audio['asc']=(df_audio.asd==11)

audio.append('sex_x')

X, y, vpn = myml.prepare_for_classify(df_audio, audio)
pr_rf_audio, pr_svm_audio, y_audio, test_index_audio=myml.classification(X, y, 'all')


### Total

In [ ]:
features=['AU06_r', 'AU12_r', 'AU04_r', 'AU09_r', 'AU15_r',
          'AU06_c', 'AU12_c', 'AU04_c', 'AU09_c', 'AU15_c', 
          'gaze_angle_x', 'gaze_angle_y']

df, new_features=myml.calc_features(df, features)
df_total=pd.merge(df, df_audio, on='vpn').reset_index()

new_features.append('sex')
new_features=new_features+audio

X, y, vpn = myml.prepare_for_classify(df_total, new_features)
pr_rf_all, pr_svm_all, y_all, test_index_all=myml.classification(X, y, 'all')

## Prediction based on AQ

In [ ]:
asq=df.groupby('vpn').mean().dropna(subset=['asq'])
X, y, vpn = myml.prepare_for_classify(asq, ['asq', 'sex'])
pr_rf_aq, pr_svm_aq, y_true_aq, test_index_aq=myml.classification(X, y, 'asq')
#lr_pr, y_true_lr, test_index=myml.threshold(X_asq[:, np.newaxis], y_asq)

## Prediction based on Clinical View 

In [ ]:
experts=mycharite.load_experts(df)
X, y, vpn = myml.prepare_for_classify(experts, ['Value', 'sex'])
pr_rf_clinical, pr_svm_clinical, y_true_clinical, test_index_clinical=myml.classification(X, y, 'Clinical')

## Combination of Features and AQ

In [ ]:
features=['AU06_r', 'AU12_r', 'AU04_r', 'AU09_r', 'AU15_r',
          'AU06_c', 'AU12_c', 'AU04_c', 'AU09_c', 'AU15_c', 
          'gaze_angle_x', 'gaze_angle_y']

df, new_features=myml.calc_features(df, features)
df_total=pd.merge(df, df_audio, on='vpn').reset_index()

new_features.append('sex')
new_features=new_features+audio

all_and_aq=new_features.append('asq')
X, y, vpn = myml.prepare_for_classify(df_total, all_and_aq)
pr_rf_all_plusAQ, pr_svm_all_plusAQ, y_all_plusAQ, test_index_all_plusAQ=myml.classification(X, y, 'all_plus_aq')

## Combination of Features and Clinical View

In [ ]:
features=['AU06_r', 'AU12_r', 'AU04_r', 'AU09_r', 'AU15_r',
          'AU06_c', 'AU12_c', 'AU04_c', 'AU09_c', 'AU15_c', 
          'gaze_angle_x', 'gaze_angle_y']

df, new_features=myml.calc_features(df, features)
df_total=pd.merge(df, df_audio, on='vpn').reset_index()

new_features.append('sex')
new_features=new_features+audio

all_and_clinic=new_features.append('Value')
df_total_clinical=pd.merge(df_total, experts, on='vpn').reset_index()

X, y, vpn = myml.prepare_for_classify(df_total_clinical, all_and_clinic)
pr_rf_all_plusClinic, pr_svm_all_plusClinic, y_all_plusClinic, test_index_all_plusClinic=myml.classification(X, y, 'all_plus_clinic')

### Graphics

In [ ]:
myml.features_roc_graph(y_true_total, y_occ, y_int, y_gaze, y_true_face, y_true_audio,
                       pr_rf_total[:,1], pr_svm_total[:,1],
                       pr_svm_occ[:,1], pr_rf_occ[:,1],  
                       pr_svm_int[:,1], pr_rf_int[:,1], 
                       pr_svm_gaze[:,1], pr_rf_gaze[:,1],
                       pr_svm_face[:,1], pr_rf_face[:,1],
                       pr_rf_audio[:,1], pr_svm_audio[:,1],
                       '(all)', '(occ)', '(int)', '(gaze)', '(face)', '(audio)', 'Feature_Comparison')

#### Correlation of Prediction based on Features and based on AQ (not correlated -> additional value of features)

In [ ]:
print ('Corrrelation of Prediction and Diagnosis')
print (asq>0)
plt.close()
y_true_new=y_true[asq>0]
tree=pr_rf[asq>0,1]
aq= lr_pr[:,1]
print (scipy.stats.pointbiserialr(pr_rf[:,1], y_true))

print ('Correlation of Prediction based on Features and based on AQ (seperated by NT and ASD)')
tree_asc=tree[y_true_new==True]
aq_asc=aq[y_true_new==True]

print (scipy.stats.normaltest(tree_asc))
print (scipy.stats.spearmanr(tree_asc, aq_asc))

tree_nt=tree[y_true_new==False]
aq_nt=aq[y_true_new==False]
print (scipy.stats.normaltest(tree_nt))
print (scipy.stats.spearmanr(tree_nt, aq_nt))

# Regression (ADOS and ADI-R)

In [ ]:
df_Reg=mycharite.ados_charite(df)

In [ ]:
#df_Reg, action_r_Reg, action_c_Reg, gaze_Reg =prep.load('charite', 'lab')
#df_Reg=prep.aq_charite(df_Reg, 'lab')
#df_Reg=prep.exclude_outlier(df_Reg)
#df_Reg, start_Reg, end_Reg=prep.get_convparts(df_Reg)
#prep.check(df_Reg)

#df=ados_charite(df)

print ('NT ' + str(sum(df_Reg.groupby('vpn').mean().asc==0)))
print ('ASC ' + str(sum(df_Reg.groupby('vpn').mean().asc==1))) 


#df[df.adir_social>0].groupby('vpn').mean().reset_index()


#calculation of features for both datasets
#df_Reg, features_part_Reg=subfeatures(df_Reg, action_r_Reg, action_c_Reg)

In [ ]:
plt.scatter(df_Reg[df_Reg.asc==1].groupby('vpn').adir_social.mean(), df_Reg[df_Reg.asc==1].groupby('vpn').asq.mean())
plt.show()

plt.figure(figsize=(12,8))
plt.scatter(df_Reg[df_Reg.asc==1].groupby('vpn').ados_total.mean(), df_Reg[df_Reg.asc==1].groupby('vpn').asq.mean())
plt.yticks(fontsize=16)    
plt.xticks(fontsize=16)  
plt.xlabel('ADOS', fontsize=18)
plt.ylabel('AQ', fontsize=18)
plt.savefig('ADOSandAQ.png')

### ADOS

In [ ]:
X_ados_rf, rescaledX_ados_rf, y_ados_rf, diagnosis = myreg.prepare_for_regress_RF(df_Reg, df_audio, 'charite', 'ADOS', features_part_Reg)
X_ados_rf_social, rescaledX_ados_rf_social, y_ados_rf_social, diagnosis = myreg.prepare_for_regress_RF(df_Reg, df_audio,'charite', 'ADOS_social', features_part_Reg)
X_ados_rf_communication, rescaledX_ados_rf_communication, y_ados_rf_communication, diagnosis = myreg.prepare_for_regress_RF(df_Reg, df_audio,'charite', 'ADOS_communication', features_part_Reg)

In [ ]:
## Prediction of ADOS - Social
print ('social')
y_pred_svr_ADOS_social, y_pred_tree_ADOS_social, y_true_ADOS_social, y_base_ADOS_social, tree_ADOS_social, svr_ADOS_social=myreg.regression(X_ados_rf_social, y_ados_rf_social, name='ADOS_social')

## Prediction of ADOS - Communication
print ('communication')
y_pred_svr_ADOS_communication, y_pred_tree_ADOS_communication, y_true_ADOS_communication, y_base_ADOS_communication, tree_ADOS_communication, svr_ADOS_communication=myreg.regression(X_ados_rf_communication, y_ados_rf_communication, name='ADOS_communication')

## Prediction of ADOS - Total
print ('total')
y_pred_svr_ADOS, y_pred_tree_ADOS, y_true_ADOS, y_base_ADOS, tree_ADOS, svr_ADOS=myreg.regression(X_ados_rf, y_ados_rf, name='ados')

In [ ]:
myreg.reg_cor(x=y_pred_svr_ADOS, y=y_true_ADOS, x_label='Predicition of SVR', y_label='ADOS Value')
myreg.reg_cor(x=y_pred_tree_ADOS, y=y_true_ADOS, x_label='Predicition of RF', y_label='ADOS Value')

In [ ]:
### ADOS

sns.regplot(df_Reg[df_Reg.ados_total>0].groupby('vpn').mean().asq, df_Reg[df_Reg.ados_total>0].groupby('vpn').mean().ados_total)
plt.show()

sns.regplot(df_Reg[df_Reg.ados_total>0].groupby('vpn').mean().asq, df_Reg[df_Reg.ados_total>0].groupby('vpn').mean().ados_social)
plt.show()

plt.scatter(df_Reg[df_Reg.ados_total>0].groupby('vpn').mean().asq, df_Reg[df_Reg.ados_total>0].groupby('vpn').mean().ados_commu)
plt.show()

### ADI-R

In [ ]:
X_adi_rf_social, rescaledX_adi_rf_social, y_adi_rf_social, diagnosis = myreg.prepare_for_regress_RF(df_Reg, df_audio, 'charite', 'ADI', features_part)

In [ ]:
print ('social')
y_pred_svr_ADI_social, y_pred_tree_ADI_social, y_true_ADI_social, y_base_ADI_social, tree_ADI_social, svr_ADI_social=myreg.regression(X_adi_rf_social, y_adi_rf_social, name='ADI')

## Prediction of ADI Social based on AQ
print ('social (based on AQ)')
y_pred_svr_AQ, y_pred_tree_AQ, y_true_AQ, y_base_AQ, tree_AQ, svr_AQ=myreg.regression(diagnosis[diagnosis>0].reshape(-1, 1), y_adi_rf_social[diagnosis>0], name='ADI')

In [ ]:
myreg.evaluate(y_pred_svr_ADI_social[diagnosis>0], y_pred_tree_ADI_social[diagnosis>0], y_adi_rf_social[diagnosis>0], y_pred_svr_AQ)

In [ ]:
myreg.evaluate(y_pred_svr_ADI_social[diagnosis>0], y_pred_tree_ADI_social[diagnosis>0], y_adi_rf_social[diagnosis>0], y_pred_svr_AQ)
print ('ADI Prediction vs. True Value')
plt.scatter(y_pred_tree_ADI_social[diagnosis>0], y_adi_rf_social[diagnosis>0])
print (stats.spearmanr(y_pred_tree_ADI_social, y_adi_rf_social))
plt.show()

In [ ]:
plt.savefig('ADI_WTAS')